# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [104]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.
'''

## 2. Generate the mathematical model

In [121]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [122]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location Problem

### Sets and Indices:

*   **I**: Set of customers, indexed by i
*   **J**: Set of potential facility locations, indexed by j

### Parameters:

*   **f_j**: Fixed annual activation cost for facility j
*   **c_ij**: Transportation cost per unit of demand for servicing customer i from facility j
*   **d_i**: Annual demand of customer i 
*   **u_j**: Maximum annual service volume capacity of facility j

### Decision Variables:

*   **x_ij**: Binary variable that equals 1 if customer i is assigned to facility j, and 0 otherwise. 
*   **y_j**: Binary variable that equals 1 if facility j is opened, and 0 otherwise.

### Objective Function: 

Minimize the total cost, which includes facility activation costs and transportation costs:

```
Minimize Z = ∑_(j∈J) f_j * y_j + ∑_(i∈I) ∑_(j∈J) c_ij * d_i * x_ij
```

### Constraints:

1.  **Demand Satisfaction**: Each customer must be assigned to exactly one open facility:

    ```
    ∑_(j∈J) x_ij = 1, ∀ i ∈ I
    ```

2.  **Facility Capacity**: The total demand assigned to a facility cannot exceed its capacity:

    ```
    ∑_(i∈I) d_i * x_ij ≤ u_j * y_j, ∀ j ∈ J
    ```
3.  **Facility Activation**: A customer can only be assigned to an open facility:

    ```
    x_ij ≤ y_j, ∀ i ∈ I, ∀ j ∈ J 
    ```

4.  **Binary Variables**:

    ```
    x_ij ∈ {0,1}, ∀ i ∈ I, ∀ j ∈ J
    y_j ∈ {0,1}, ∀ j ∈ J
    ```

This model is a mixed-integer linear program (MILP) that can be solved using various optimization software packages. 


## 3. Generate the pyomo code

In [123]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [124]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
I = {1, 2, 3}  # Set of customers
J = {1, 2}  # Set of potential facility locations
f = {1: 1000, 2: 1200}  # Fixed annual activation cost for each facility
c = {
    (1, 1): 5,
    (1, 2): 7,
    (2, 1): 6,
    (2, 2): 4,
    (3, 1): 8,
    (3, 2): 3,
}  # Transportation cost per unit of demand
d = {1: 100, 2: 150, 3: 200}  # Annual demand of each customer
u = {1: 350, 2: 400}  # Maximum annual service volume capacity of each facility

# Create a Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.J, initialize=f)
model.c = pyo.Param(model.I, model.J, initialize=c)
model.d = pyo.Param(model.I, initialize=d)
model.u = pyo.Param(model.J, initialize=u)

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.y = pyo.Var(model.J, domain=pyo.Binary)

# Objective Function
def total_cost(model):
    return sum(model.f[j] * model.y[j] for j in model.J) + sum(
        model.c[i, j] * model.d[i] * model.x[i, j] for i in model.I for j in model.J
    )


model.total_cost = pyo.Objective(rule=total_cost, sense=pyo.minimize)

# Constraints
def demand_satisfaction(model, i):
    return sum(model.x[i, j] for j in model.J) == 1


model.demand_constraint = pyo.Constraint(model.I, rule=demand_satisfaction)

def facility_capacity(model, j):
    return sum(model.d[i] * model.x[i, j] for i in model.I) <= model.u[j] * model.y[j]


model.capacity_constraint = pyo.Constraint(model.J, rule=facility_capacity)

def facility_activation(model, i, j):
    return model.x[i, j] <= model.y[j]


model.activation_constraint = pyo.Constraint(model.I, model.J, rule=facility_activation)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
print("Optimal Cost:", model.total_cost())
for j in model.J:
    if model.y[j].value == 1:
        print(f"Facility {j} is opened")
        for i in model.I:
            if model.x[i, j].value == 1:
                print(f"\tCustomer {i} is assigned to facility {j}")
``` 
**Explanation of Sample Data:**

*   **Sets I and J:**  These sets define the customers and potential facility locations. 
*   **Parameters f, c, d, and u:** These parameters provide the fixed costs, transportation costs, customer demands, and facility capacities, respectively. You'll need to replace these with your actual data. 

**Note:** The solver used in this code is 'glpk', which is a free and open-source solver. You can choose other solvers based on your preference and licensing.


## 4. Input problem data and try running the generated code

In [126]:
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
I = {1, 2, 3,4,5}  # Set of customers
J = {1, 2,3}  # Set of potential facility locations
f = {1: 1000, 2: 1000, 3: 1000}  # Fixed annual activation cost for each facility
c = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}
     # Transportation cost per unit of demand
d = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # Annual demand of each customer
u = {1: 500, 2: 500, 3: 500} # Maximum annual service volume capacity of each facility

# Create a Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.J, initialize=f)
model.c = pyo.Param(model.I, model.J, initialize=c)
model.d = pyo.Param(model.I, initialize=d)
model.u = pyo.Param(model.J, initialize=u)

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.y = pyo.Var(model.J, domain=pyo.Binary)

# Objective Function
def total_cost(model):
    return sum(model.f[j] * model.y[j] for j in model.J) + sum(
        model.c[i, j] * model.d[i] * model.x[i, j] for i in model.I for j in model.J
    )


model.total_cost = pyo.Objective(rule=total_cost, sense=pyo.minimize)

# Constraints
def demand_satisfaction(model, i):
    return sum(model.x[i, j] for j in model.J) == 1


model.demand_constraint = pyo.Constraint(model.I, rule=demand_satisfaction)

def facility_capacity(model, j):
    return sum(model.d[i] * model.x[i, j] for i in model.I) <= model.u[j] * model.y[j]


model.capacity_constraint = pyo.Constraint(model.J, rule=facility_capacity)

def facility_activation(model, i, j):
    return model.x[i, j] <= model.y[j]


model.activation_constraint = pyo.Constraint(model.I, model.J, rule=facility_activation)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
print("Optimal Cost:", model.total_cost())
for j in model.J:
    if model.y[j].value == 1:
        print(f"Facility {j} is opened")
        for i in model.I:
            if model.x[i, j].value == 1:
                print(f"\tCustomer {i} is assigned to facility {j}")

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
Optimal Cost: 6620.0
Facility 1 is opened
	Customer 1 is assigned to facility 1
	Customer 2 is assigned to facility 1
Facility 2 is opened
	Customer 3 is assigned to facility 2
Facility 3 is opened
	Customer 4 is assigned to facility 3
	Customer 5 is assigned to facility 3


## 5. Correct the code to verify model viability (optional)

In [ ]:
#The model formulation above is unexpected, however still correct!

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [119]:
print(response.text)

## Mathematical Optimization Model for Facility Location

### Sets and Indices:

*   **I:** Set of customers (i ∈ I)
*   **J:** Set of potential facility locations (j ∈ J)

### Parameters:

*   **f<sub>j</sub>:** Fixed annual activation cost for facility at location j
*   **c<sub>ij</sub>:** Transportation cost per unit of demand for serving customer i from facility j
*   **d<sub>i</sub>:** Annual demand of customer i
*   **u<sub>j</sub>:** Maximum annual service volume capacity of facility at location j 

### Decision Variables:

*   **x<sub>ij</sub>:** Fraction of customer i's demand served by facility at location j (0 ≤ x<sub>ij</sub> ≤ 1)
*   **y<sub>j</sub>:** Binary variable indicating whether a facility is opened at location j (y<sub>j</sub> = 1 if open, 0 otherwise)

### Objective Function: 

Minimize the total cost (fixed activation costs + transportation costs)

```
Minimize Z = ∑_{j∈J} f_j * y_j + ∑_{i∈I} ∑_{j∈J} c_ij * d_i * x_ij
```

### Constraints:

1.  **Demand Satisfac

In [120]:
print(response2.text)

```python
# Import Pyomo library
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sample Data (replace with your actual data)
I = {1, 2, 3}  # Set of customers
J = {1, 2}  # Set of potential facility locations
f = {1: 1000, 2: 1200}  # Fixed annual activation cost for each facility
c = {(1, 1): 5, (1, 2): 7, (2, 1): 6, (2, 2): 4, (3, 1): 8, (3, 2): 3}  # Transportation cost per unit of demand
d = {1: 200, 2: 300, 3: 250}  # Annual demand of each customer
u = {1: 500, 2: 600}  # Maximum annual service volume capacity of each facility

# Define sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Define parameters
model.f = Param(model.J, initialize=f)
model.c = Param(model.I, model.J, initialize=c)
model.d = Param(model.I, initialize=d)
model.u = Param(model.J, initialize=u)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))  # Fraction of customer demand served by each facility
model.y = Var(model.J, within=Binary)  #